# Recommender System for movies

In this notebook we will create a Recommender System for movies using a Restricted Boltzman Machine. 
The Restricted Boltzman Machine is a Generative Model, with undirectional edges, composed of visible and hidden nodes.
We can see the structure of the network in this picture:

<img src="restrictedBoltzman.png">

In particular, in this model we will use the following datasets:

https://grouplens.org/datasets/movielens/1m/

https://grouplens.org/datasets/movielens/100k/

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
 * 100,000 ratings (1-5) from 943 users on 1682 movies. 
 * Each user has rated at least 20 movies. 
 * Simple demographic info for the users (age, gender, occupation, zip)


## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Dataset

In [3]:
# Movies Data
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = ['MovieID', 'Title', 'Genre' ])
movies.head()

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# Users Data
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = ['UserID', 'Gender', 'Age', 'Job', 'ZipCode' ])
users.head()

,UserID,Gender,Age,Job,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
# Ratings Data
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = ['UserID', 'MovieID', 'Rating', 'Timestamp' ])
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Prepare Training Set and Test set

We will use an holdout train test split. In particular we will use 80% of the data as training set and the remaining 20 % as test set

In [6]:
# Train set
training_set = pd.read_csv('ml-100k/u1.base', sep = '\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [7]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79999 entries, 0 to 79998
Data columns (total 4 columns):
1            79999 non-null int64
1.1          79999 non-null int64
5            79999 non-null int64
874965758    79999 non-null int64
dtypes: int64(4)
memory usage: 2.4 MB


In [8]:
# Convert the df into an array
training_set = np.array(training_set, dtype = 'int')
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [9]:
# Test set
test_set = pd.read_csv('ml-100k/u1.test', sep = '\t')
test_set.head()

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [10]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 4 columns):
1            19999 non-null int64
6            19999 non-null int64
5            19999 non-null int64
887431973    19999 non-null int64
dtypes: int64(4)
memory usage: 625.0 KB


In [11]:
test_set = np.array(test_set, dtype = 'int')
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [12]:
# Getting the number of users and movies
n_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
n_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

Now we will convert the data into an array whith users in line and movies in columns. For this reason we extracted the number of users and movies. The values will be the ratings. In particular we will create a list of list, in particular a list of movie ratings for each user.

In [13]:
def convert(data):
    new_data = []
    for id_users in range(1, n_users + 1):
        id_movies = data[:, 1][data[:,0] == id_users] # Select all the movies ID that corresponds to the user id
        id_ratings = data[:, 2][data[:,0] == id_users] # Select all the ratings ID that corresponds to the user id
        ratings = np.zeros(n_movies)
        ratings[id_movies - 1] = id_ratings# id_movies start at 0
        new_data.append(list(ratings))
    return new_data

In [14]:
training_set = convert(training_set)
test_set = convert(test_set)

Now we convert the data into Torch Tensor 

In [15]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Convert the ratings into binary ratings: 1(liked) or 0(not liked)